<!--  Title slide -->
<h1><strong>Module 1: Intro to SBI (module name) </strong></h1>
<h2> <em> Simulation-based Inference for scientific discovery </em></h2>

<h3>Instructor (your name)</h3>

In [ ]:
# imports
import torch
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

<!--  Theory slide -->
<!-- ## corresponds to section name, ### to subsection-->
## Simulators for science (section name)
### Simulators (sub-section name)
- Let's say, we are investigating sine waves
- Interested in finding amplitude $\theta$ given an observed wave $x$
    - Posterior $p(\theta | x)$
- We can implicitly define likelihood $p(x | \theta)$ by building simulator
- Let's have a look at this simulator

### Example simulator - Practical 🛠️

- With our theoretic background on sine waves, we can now explore practical examples

In [ ]:
# code slides for practical

def wave_simulator(theta, plot=False):
    """ f(x | theta) = theta * sin(x) -- intented to be used in interactive widget. """
    xs = np.linspace(0,10,100) # x values
    observation = theta * torch.sin(torch.Tensor(xs))  + torch.randn(len(xs)) * 0.1 # y values
    if plot: # make it usable in interactive plot -- boilerplate for functions to be used in interactive widgets
        plt.plot(observation,label=f'$\\theta={theta}$',color='#FF1053')
        plt.ylim(-5, 5)
        plt.legend()
        plt.show()

    return observation

# code, text that belongs together (i.e. because of shared variables) can be navigated downwards in slide mode.
# This is done, by selecting Sub-Slide in the dropdown menu on the top right. 
# Will keep this whole practical as Sub-Slides

You can play around with the simulator and produce different observations by adjusting the $\theta$ parameter.

In [ ]:
# boilerplate for interactive widgets
simulation = interactive(wave_simulator, theta=(-2.0, 2.0), plot=True)
simulation

- Use simulator to produce many observations $x$ and form synthetic dataset
- Choose sample size $n$ 
- Define uniform distribution to represent prior belive about what $\theta$ could be
- Let `sbi` do the work

In [ ]:
# initiate samples 
num_samples, prior_lower, prior_upper = 500, -1, 1 # try out different values and see which work well and why

samples = np.zeros((num_samples, len(np.linspace(0,10,100))))

# create prior
prior = utils.BoxUniform(low=prior_lower*torch.ones(1), high=prior_upper*torch.ones(1))

# run simulator and infer 
posterior = infer(wave_simulator, prior, method='SNPE', num_simulations=num_samples)

- Great! We now have a posterior over $\theta$
- Let's have a look whether it is good
- Use simulator to create another example observation
- See whether probability mass accumulates at chosen $\theta$

In [ ]:
 simulation

In [ ]:
# the simulation.result is boilerplate to obtain the returned values of the function used in the widget
sample = posterior.sample((10000,), x=simulation.result) 
_ = analysis.pairplot(sample, limits=[[-2,2],[-2,2],[-2,2]], figsize=(6,6))
plt.show()

Are you satisfied with the conditional posterior? If not, maybe, try out different hyperparameters.